# Home Task: 2D Convolution

In [38]:
%%writefile hometask.cu
#include <cuda.h>
#include <stdio.h>
#include <iostream>

using namespace std;

__constant__ int k = 3;
__constant__ int filter[9];

__global__ void function(int *input, int *output, int n, int m){
  int globalIdx = blockIdx.x * blockDim.x + threadIdx.x;
  if(globalIdx < n * m){
    int res = 0;

    for(int i = max(0, globalIdx / m - 1); i <= min(n - 1, globalIdx / m + 1); i++){
      for(int j = max(0, globalIdx % m - 1); j <= min(m - 1, globalIdx % m + 1); j++){
        int kx = (i - globalIdx / m) + 1;
        int ky = (j - globalIdx % m) + 1;
        res += input[i * m + j] * filter[kx * k + ky];
      }
    }

    output[globalIdx] = res;
  }
}

void print_matrix(int *matrix, int n, int m){
  for(int i = 0; i < n; i++){
    for(int j = 0; j < m; j++){
      cout << matrix[i * m + j] << " ";
    } cout << endl;
  }
}

int main(){
  int n = 3;
  int m = 3;
  int nxm = n * m;
  int size = nxm * sizeof(int);
  int *host_input = (int *) malloc(size);
  int *host_output = (int *) malloc(size);
  int host_filter[9];
  for(int i = 0; i < 3 ; i++){
    int v = 1;
    for(int j = 0; j < 3 ; j++){
      host_filter[i * 3 + j] = v;
      v--;
    }
  }

  for(int i = 0; i < nxm; i++){
    host_input[i] = i + 1;
  }

  int *device_input;
  int *device_output;
  cudaMalloc(&device_input, size);
  cudaMalloc(&device_output, size);
  cudaMemcpyToSymbol(filter, host_filter, sizeof(host_filter));
  cudaMemcpy(device_input, host_input, size, cudaMemcpyHostToDevice);

  int blocksize = 256;
  int gridsize = (nxm + blocksize - 1) / blocksize;
  function<<<gridsize, blocksize>>>(device_input, device_output, n, m);
  cudaMemcpy(host_output, device_output, size, cudaMemcpyDeviceToHost);

  cout << "INPUT MATRIX: " << endl;
  print_matrix(host_input, n, m);

  cout << "FILTER: " << endl;
  print_matrix(host_filter, 3, 3);

  cout << "OUTPUT MATRIX: " << endl;
  print_matrix(host_output, n, m);
  return 0;
}

Overwriting hometask.cu


In [39]:
!nvcc hometask.cu -o hometask
!./hometask

INPUT MATRIX: 
1 2 3 
4 5 6 
7 8 9 
FILTER: 
1 0 -1 
1 0 -1 
1 0 -1 
OUTPUT MATRIX: 
-7 -4 7 
-15 -6 15 
-13 -4 13 
